# Задание 2. Итерационные методы решения линейных систем

In [46]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple, Union
import warnings
warnings.filterwarnings('ignore')

In [47]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """
        Calculates Matrix norm using np.linalg.norm
        
        ## Returns
        Matrix norm using formulae max(sum(abs(x), axis=0)) either as `string` or a `float`
        """
        return np.linalg.norm(self, 1)


def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg, np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)

    ## Parameters
    `A`: матрица коэффициентов размерности `n`x`n`
    `B`: столбец свободных членов размерности `n`x`1`
    `n`: размерность

    ## Returns
    `X`: столбец, такой что AX ≈ B
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n):
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

In [48]:
n = 4
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, n) * 10)

In [49]:
print_tex(A, r'\times', B, '=', A @ B)

$$ \begin{pmatrix} 1.415513 & 3.127778 & 2.407085 & 3.767383 \\ 8.832330 & 8.668505 & 5.066829 & 6.090286 \\ 7.621917 & 6.988894 & 1.884004 & 4.221194 \\ 7.387296 & 8.978367 & 7.669475 & 0.093020 \end{pmatrix} \times \begin{pmatrix} 1.672984 & 5.290744 & 3.879802 & 1.755054 \\ 7.607616 & 3.433011 & 7.968475 & 4.203982 \\ 7.847937 & 5.015478 & 6.038371 & 8.221748 \\ 7.171213 & 0.124265 & 2.941688 & 8.720751 \end{pmatrix} = \begin{pmatrix} 72.070420 & 30.767646 & 56.032866 & 68.278278 \\ 164.161897 & 102.658046 & 151.853568 & 146.713519 \\ 110.976798 & 74.292288 & 109.056108 & 95.059857 \\ 141.519417 & 108.384760 & 146.789904 & 114.577696 \end{pmatrix} $$

$$ \begin{pmatrix} 1.415513 & 3.127778 & 2.407085 & 3.767383 \\ 8.832330 & 8.668505 & 5.066829 & 6.090286 \\ 7.621917 & 6.988894 & 1.884004 & 4.221194 \\ 7.387296 & 8.978367 & 7.669475 & 0.093020 \end{pmatrix} \times \begin{pmatrix} 1.672984 & 5.290744 & 3.879802 & 1.755054 \\ 7.607616 & 3.433011 & 7.968475 & 4.203982 \\ 7.847937 & 5.015478 & 6.038371 & 8.221748 \\ 7.171213 & 0.124265 & 2.941688 & 8.720751 \end{pmatrix} = \begin{pmatrix} 72.070420 & 30.767646 & 56.032866 & 68.278278 \\ 164.161897 & 102.658046 & 151.853568 & 146.713519 \\ 110.976798 & 74.292288 & 109.056108 & 95.059857 \\ 141.519417 & 108.384760 & 146.789904 & 114.577696 \end{pmatrix} $$